In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\home laptop\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
from numpy.random import seed
seed(1)
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_slogg']]
selected_features = selected_features[selected_features['koi_disposition']!='CANDIDATE']
# selected_features['koi_disposition'] = selected_features['koi_disposition'].map({'CONFIRMED': 1, 'CANDIDATE': 2, 'FALSE POSITIVE':3})
selected_features

,koi_disposition,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_slogg
0,CONFIRMED,0.586,4.50700,874.8,2.83,443,9.11,25.8,4.467
1,FALSE POSITIVE,0.969,1.78220,10829.0,14.60,638,39.30,76.3,4.544
2,FALSE POSITIVE,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,4.564
3,CONFIRMED,0.701,1.65450,603.3,2.75,1406,926.16,40.9,4.438
4,CONFIRMED,0.762,3.14020,686.0,2.77,1160,427.65,40.2,4.486
...,...,...,...,...,...,...,...,...,...
6983,FALSE POSITIVE,2.441,77.98300,1583.8,534.47,1006,242.54,171.5,3.508
6986,FALSE POSITIVE,0.765,4.80600,87.7,1.11,929,176.40,8.4,4.296
6987,FALSE POSITIVE,1.252,3.22210,1579.2,29.35,2088,4500.53,453.3,4.529
6989,FALSE POSITIVE,0.147,0.86500,103.6,1.07,2218,5713.41,12.3,4.447


In [6]:
# Read the training data into a dataframe
X = selected_features[['koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr', 'koi_slogg']]
y = selected_features['koi_disposition']

In [7]:
#columns
feature_names = selected_features.columns[1:9]
target = selected_features.columns[:1]
target_names = ["CONFIRMED", "FALSE POSITIVE"]
print(feature_names,target_names)

Index(['koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq',
       'koi_insol', 'koi_model_snr', 'koi_slogg'],
      dtype='object') ['CONFIRMED', 'FALSE POSITIVE']


In [8]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
from sklearn.metrics import classification_report

# Create a Train Test Split

In [9]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, test_size=0.2)

In [10]:
X_train.head()

,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_slogg
2067,0.995,3.4303,467.3,7.84,1195,482.18,74.3,4.005
991,0.216,3.9870,690.6,2.11,652,42.82,22.4,4.557
772,0.927,2.4657,7589.6,10.68,591,28.80,148.1,4.479
2883,0.389,1.3911,339.6,1.61,1769,2325.64,57.0,4.466
2654,0.013,2.6903,1026.2,2.94,825,109.61,59.1,4.506


# Pre-processing
Scale the data using the MinMaxScaler and perform some feature selection

In [11]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [12]:
# Scale your data
# Create a MinMaxScaler model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model

In [13]:
# Fit to the training data and validate with the test data
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [14]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6606174876266793
Testing Data Score: 0.6606974552309143


In [15]:
predictions = model.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
224,FALSE POSITIVE,FALSE POSITIVE
5175,FALSE POSITIVE,FALSE POSITIVE
710,FALSE POSITIVE,FALSE POSITIVE
5032,FALSE POSITIVE,FALSE POSITIVE
529,FALSE POSITIVE,CONFIRMED
...,...,...
4909,FALSE POSITIVE,FALSE POSITIVE
1806,FALSE POSITIVE,FALSE POSITIVE
5912,FALSE POSITIVE,FALSE POSITIVE
1795,FALSE POSITIVE,CONFIRMED


# Hyperparameter Tuning¶
Use GridSearchCV to tune the model's parameters

In [16]:
model.get_params()

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto_deprecated',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [17]:
# Train the model with GridSearch
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [18]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=1, gamma=0.0001 ...............................................


C:\Users\Home Laptop\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.661, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.661, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.661, total=   0.2s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.661, total=   0.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.661, total=   0.2s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.661, total=   0.1s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.661, total=   0.2s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.661, total=   0.1s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.661, total=   0.2s
[CV] C=1, gamma=0.005 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    7.5s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [19]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.7612538298373792


In [20]:
predictions = grid.predict(X_test_scaled)
predictions

array(['FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', ...,
       'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [21]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["Exoplanet", "Dust"]))

              precision    recall  f1-score   support

   Exoplanet       0.65      0.68      0.67       360
        Dust       0.83      0.81      0.82       701

    accuracy                           0.77      1061
   macro avg       0.74      0.75      0.75      1061
weighted avg       0.77      0.77      0.77      1061



In [22]:
import joblib
filename = 'svm.h5'
joblib.dump(model, filename)

['svm.h5']